# Ambiguity Plot

Goal of this notebook is to quickly visualize the results of the CAF (Cross Ambiguity Function).

## Setup

First, run some imports of common modules. Also note the import of `pluto_sdr_pr` from this repository. 

In [ ]:
%matplotlib widget

import numpy as np

import matplotlib.cm
import matplotlib.pyplot as plt
from matplotlib import animation

import sys
sys.path.append('../')
import pluto_sdr_pr.ioutils
import pluto_sdr_pr.processing

## Define Constants

Define some common constants for subsequent use.

In [ ]:
c = 3e8 # in m/s
cpi = 0.1 # in seconds
Fs = 2.88e6 # in Hz
fc = 625e6 # in Hz

## Read Samples

Read a few samples from IQ recording. The recording is too big to include it in this repository. Thus the file has to be sourced some other way.

In [ ]:
num_samples = int(cpi * Fs)
with open('../data/pluto_b_surv.2021-07-27T15_24_21_819.sdriq', 'rb') as fid:
    n, _ = pluto_sdr_pr.ioutils.read_sdriq_samples(fid, num_samples)

## Calculate Ambiguity

Calculate blind ambiguity of recorded signal with itself. Due to RAM constraints of the current `direct_ambiguity` implementation, processing has to be performed in chunks of `step_size`.

In [ ]:
max_distance = int(50e3)
max_delay = int(max_distance / c * Fs)
max_speed = 280 # m/s
max_doppler = int(max_speed * cpi * fc / c)

%time amb = pluto_sdr_pr.processing.fast_ambiguity(max_delay, 2 * max_doppler + 1, n, n)

## 2d Line Plot

Plot result of blind correlation on 2d line plot. The lines for each doppler bin are layered ontop each other.

In [ ]:
fig = plt.figure(figsize=(20, 5))
ax1 = fig.add_subplot(111)
ax1.set_xlabel("bistatic range [km]")
ax1.set_ylabel("correlation [dB]")
ax1_xticks = np.linspace(0, max_distance / 1000, 22)
ax1.set_xticks(ax1_xticks)
ax1.plot(np.arange(max_delay) / Fs * c / 1000, 10 * np.log10(np.abs(amb)))

ax2 = ax1.twiny()
ax2_xticks = np.linspace(0, max_distance / 1000, 22)
ax2.set_xticks(ax2_xticks)
ax2.set_xticklabels(map(lambda tick: f"{(tick * 1e9) / 3e8:.0f}", ax2_xticks))
ax2.set_xlim(ax1.get_xlim())
ax2.set_xlabel("time [µsec]")
plt.title(f"Correlation of {cpi}sec CPI, Doppler: {1}", pad=40);

ax1.grid()
plt.grid(axis='x', alpha=0.5, linestyle='--')


## 3d Surface Plot

Plot as 3d surface plot, with bistatic range, bistatic velocity and calculated correlation as the three axis.

In [ ]:
X, Y = np.meshgrid(np.arange(amb.shape[0]), np.arange(amb.shape[1]))
Z = 10 * np.log10(np.abs(amb))[X, Y]

fig, ax = plt.subplots(subplot_kw={"projection": "3d"}, figsize=(10, 10))
ax.plot_surface(X, Y, Z, cmap=matplotlib.cm.Spectral);

## Range/Doppler Map

Plot ambiguity as 2d image, color-coding spots of high correlation. This representation is commonly known as _Range/Doppler Map_.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
ax.set_title("Range/Doppler Map")
ax.set_xlabel("bistatic range [km]")
ax.set_ylabel("bistatic velocity [m/s]")
xticks = np.linspace(0, 50e3, 6, endpoint=True)
ax.set_xticks(xticks / c * Fs)
ax.set_xticklabels(map(lambda x: f"{x // 1e3:.0f} km", xticks))

yticks = np.linspace(-max_speed, max_speed, 15, endpoint=True)
ax.set_yticks((yticks + max_speed) * cpi * fc / c)
ax.set_yticklabels(map(lambda y: f"{y:.0f} m/s", yticks))
plt.imshow(10 * np.log10(np.abs(amb.T)));

## GPU Accelerated Ambiguity Function

In [ ]:
ref_file_path = "../data/pluto_a_ref.2021-07-27T15_24_17_598.sdriq"
surv_file_path = "../data/pluto_b_surv.2021-07-27T15_24_21_819.sdriq"

_, hdr = pluto_sdr_pr.ioutils.read_sdriq_samples(ref_file_path, 0, 0)

sample_rate = hdr["sample_rate"]

num_samples_in_cpi = int(sample_rate * 1)
num_cpis = 10

ref_samples, _ = pluto_sdr_pr.ioutils.read_sdriq_samples(ref_file_path, int(num_samples_in_cpi * num_cpis), 0)
surv_samples, _ = pluto_sdr_pr.ioutils.read_sdriq_samples(surv_file_path, int(num_samples_in_cpi * num_cpis), 0)

amb = pluto_sdr_pr.processing.fast_ambiguity(max_delay, 2 * max_doppler + 1, ref_samples[:num_samples_in_cpi], surv_samples[:num_samples_in_cpi])
#amb = pluto_sdr_pr.processing.gpu_ambiguity(max_delay, 2 * max_doppler + 1, ref_samples, surv_samples, num_samples_in_cpi, batch_size=1)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
ax.set_title("Range/Doppler Map")
ax.set_xlabel("bistatic range [km]")
ax.set_ylabel("bistatic velocity [m/s]")
xticks = np.linspace(0, 50e3, 6, endpoint=True)
ax.set_xticks(xticks / c * Fs)
ax.set_xticklabels(map(lambda x: f"{x // 1e3:.0f} km", xticks))

yticks = np.linspace(-max_speed, max_speed, 15, endpoint=True)
ax.set_yticks((yticks + max_speed) * cpi * fc / c)
ax.set_yticklabels(map(lambda y: f"{y:.0f} m/s", yticks))
plt.imshow(10 * np.log10(np.abs(amb[:, :, 0].T)));

## Animated Range/Doppler Map

Plot ambiguity as 2d image over 10 seconds of recorded and analyzed data

In [ ]:
num_frames = 100

plt.ioff()

fig, ax = plt.subplots(figsize=(10, 5))
ax.set_title("Range/Doppler Map")
ax.set_xlabel("bistatic range [km]")
ax.set_ylabel("bistatic velocity [m/s]")
xticks = np.linspace(0, 50e3, 6, endpoint=True)
ax.set_xticks(xticks / c * Fs)
ax.set_xticklabels(map(lambda x: f"{x / 1e3:.0f} km", xticks))

yticks = np.linspace(-max_speed, max_speed, 15, endpoint=True)
ax.set_yticks((yticks + max_speed) * cpi * fc / c)
ax.set_yticklabels(map(lambda y: f"{y:.0f} m/s", yticks))

with open('../data/gqrx_20210326_161137_625000000_2880000_fc_flugplatz_longdipole.raw', 'rb') as fid:
    def render(frame):
        n = pluto_sdr_pr.ioutils.read_samples(fid, num_samples, frame * num_samples)
        amb = pluto_sdr_pr.processing.fast_ambiguity(max_delay, 2 * max_doppler + 1, n, n).T
        return (plt.imshow(10 * np.log10(np.abs(amb))),)

    anim = animation.FuncAnimation(fig, render, frames=num_frames, interval=cpi * 1e3)
    %time anim.save("range-doppler.mp4")

plt.ion();